# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [130]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [131]:
city_weather = pd.read_csv("city_weather.csv")
city_weather = city_weather.set_index("City")
# city_weather = city_weather.drop(columns = "Unnamed: 0")
city_weather_converted = city_weather.assign(Fahrenheit = lambda x: (9/5)*x['Temp']+32)
city_weather_converted.count()

Unnamed: 0    568
Lat           568
Long          568
Temp          568
Humidity      568
Cloudiness    568
Wind Speed    568
County        562
City Time     568
Fahrenheit    568
dtype: int64

In [132]:
new_city_weather = city_weather_converted.rename(columns= {"Fahrenheit": "Temp(F)"})
new_city_weather = new_city_weather.drop(columns = "Temp")
new_city_weather.head()
                                             

,Unnamed: 0,Lat,Long,Humidity,Cloudiness,Wind Speed,County,City Time,Temp(F)
City,,,,,,,,,
Bredasdorp,0,-34.5322,20.0403,88,34,3.09,ZA,2021-05-04 18:47:31,62.600
Touros,1,-5.1989,-35.4608,94,75,2.06,BR,2021-05-04 13:47:32,75.200
Busselton,2,-33.6500,115.3333,97,100,3.84,AU,2021-05-05 00:47:32,59.000
Rikitea,3,-23.1203,-134.9692,79,98,5.24,PF,2021-05-04 07:47:32,78.602
Manyana,4,-23.4000,21.7167,42,0,2.66,BW,2021-05-04 18:47:33,57.902


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [133]:
gmaps.configure(api_key=g_key)

In [134]:
locations = new_city_weather[["Lat", "Long"]]
humidity = new_city_weather["Humidity"].astype(float)
maxhumidity = humidity.max()

In [135]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating=False, max_intensity=maxhumidity,
                                point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [136]:
new_city_weather = new_city_weather.loc[(new_city_weather["Temp(F)"]>=70) & (new_city_weather["Temp(F)"]<=80)]
perfect_city_weather = new_city_weather.loc[(new_city_weather["Wind Speed"] <=10) & (new_city_weather["Cloudiness"]==0)]
perfect_city_weather

,Unnamed: 0,Lat,Long,Humidity,Cloudiness,Wind Speed,County,City Time,Temp(F)
City,,,,,,,,,
Arraial do Cabo,11,-22.9661,-42.0278,83,0,4.12,BR,2021-05-04 13:47:35,75.200
Iranshahr,53,27.2025,60.6848,50,0,2.06,IR,2021-05-04 21:16:21,75.200
Arandis,105,-22.4167,14.9667,31,0,3.16,NaN,2021-05-04 18:48:05,76.838
Oussouye,110,12.4850,-16.5469,88,0,1.03,SN,2021-05-04 16:48:07,77.000
Rosetta,111,31.4044,30.4164,50,0,1.03,EG,2021-05-04 18:48:07,75.200
São João da Barra,234,-21.6403,-41.0511,83,0,4.12,BR,2021-05-04 13:48:48,75.200
Moranbah,325,-22.0016,148.0466,55,0,1.61,AU,2021-05-05 02:49:20,71.366
Kremastí,358,36.4111,28.1189,53,0,7.72,GR,2021-05-04 19:49:31,73.400
Nísia Floresta,422,-6.0911,-35.2086,94,0,2.06,BR,2021-05-04 13:49:51,75.200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [137]:
hotel_df = pd.DataFrame(perfect_city_weather)
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,Lat,Long,Humidity,Cloudiness,Wind Speed,County,City Time,Temp(F),Hotel Name
City,,,,,,,,,,
Arraial do Cabo,11,-22.9661,-42.0278,83,0,4.12,BR,2021-05-04 13:47:35,75.200,
Iranshahr,53,27.2025,60.6848,50,0,2.06,IR,2021-05-04 21:16:21,75.200,
Arandis,105,-22.4167,14.9667,31,0,3.16,NaN,2021-05-04 18:48:05,76.838,
Oussouye,110,12.4850,-16.5469,88,0,1.03,SN,2021-05-04 16:48:07,77.000,
Rosetta,111,31.4044,30.4164,50,0,1.03,EG,2021-05-04 18:48:07,75.200,
São João da Barra,234,-21.6403,-41.0511,83,0,4.12,BR,2021-05-04 13:48:48,75.200,
Moranbah,325,-22.0016,148.0466,55,0,1.61,AU,2021-05-05 02:49:20,71.366,
Kremastí,358,36.4111,28.1189,53,0,7.72,GR,2021-05-04 19:49:31,73.400,
Nísia Floresta,422,-6.0911,-35.2086,94,0,2.06,BR,2021-05-04 13:49:51,75.200,


In [148]:
hotel_df = []
    


for x in range(len(hotel_df)):
    lat = hotel_df.loc[x]["Latitude"]
    lng = hotel_df.loc[x]["Longitude"]
    params = {
    "radius": 5000,
    "key": g_key,
    "keyword": "hotel",
    "type":"hotel",
    "location": f"{lat},{lng}"}


    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response= requested.json()
    
    
    try:
        hotel_df.append(response['results'][0]['name'])
    except (KeyError, IndexError):
        hotel_df.append("")
        

    
    

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)                            
# Display figure
fig.add_layer(markers)
fig